In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from pdb import set_trace

# Initialization

In [7]:
def calc_grad():
    for name, param in model.named_parameters():
        if name == "fc1.weight":
            print("Gradient of weight:", param.grad)

In [8]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(2, 1) 
#         self.relu = nn.ReLU()      
#         self.fc2 = nn.Linear(1, 1) 

    def forward(self, x):
        x = self.fc1(x)
#         x = self.relu(x)
#         x = self.fc2(x)
        return x

In [9]:
x = torch.tensor([
    [1.0, 3.0],
    [4.0, 1.2],
    [3.0, 2.0]
])
y = torch.tensor([1.0, 0, 1.0])

In [10]:
x.shape

torch.Size([3, 2])

In [11]:
epochs = 1

In [12]:
for epoch in range(epochs):
    for i in range(0, n,  bs):
        s = slice(i, min(n, i+bs))
        xb = x_train[s]
        yb = y_train[s].long()
        preds = model(xb)
        loss = loss_func(preds, yb)
        loss.backward()
        
        if i % 100 == 0:
            print(loss.item())
        
        with torch.no_grad():
            for l in model.layers:
                if hasattr(l, 'weight'):
                    l.weight -= l.weight.grad * lr
                    l.bias -= l.bias.grad * lr
                    l.weight.grad.zero_()
                    l.bias.grad.zero_()  

NameError: name 'n' is not defined

In [13]:
def train(inputs=x, targets=y):
    optimizer.zero_grad()
    outputs = model(inputs) # forward pass
    loss = criterion(outputs.squeeze(1), targets) # loss calculation
    loss.backward() # gradient calculation 
    optimizer.step() # weights update
#     outputs = model(inputs) # forward pass
#     new_loss = criterion(outputs, targets) # new loss calculation
#     print("New Loss after one step:", new_loss.item())

In [14]:
model = SimpleNet()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [15]:
w1 = torch.tensor([[1.0, 2.0]])
b1 = torch.tensor([1.0])

In [16]:
model.fc1.weight.data

tensor([[0.4401, 0.5435]])

In [17]:
model.fc1.bias.data

tensor([-0.2245])

In [18]:
model.fc1.weight.data = w1.clone()
model.fc1.bias.data = b1.clone()

In [19]:
model.fc1.weight.data, model.fc1.bias.data

(tensor([[1., 2.]]), tensor([1.]))

In [20]:
first_pred = model(x)
first_pred

tensor([[8.0000],
        [7.4000],
        [8.0000]], grad_fn=<AddmmBackward0>)

In [ ]:
x.shape

In [ ]:
model.fc1.weight.shape

In [21]:
def calc_loss(pred):
    res = 0
    for idx, num in enumerate(pred):
        res += (pred[idx] - y[idx])**2
    return res / len(pred)

In [22]:
calc_loss(first_pred)    

tensor([50.9200], grad_fn=<DivBackward0>)

In [23]:
first_pred.shape, y.shape

(torch.Size([3, 1]), torch.Size([3]))

In [24]:
model.fc1.weight.grad

In [25]:
train()

In [26]:
model.fc1.weight.grad, model.fc1.bias.grad

(tensor([[38.4000, 29.2533]]), tensor([14.2667]))

In [27]:
second_pred = model(x)
second_pred

tensor([[6.5957],
        [5.3703],
        [6.1203]], grad_fn=<AddmmBackward0>)

In [28]:
calc_loss(second_pred)

tensor([28.7898], grad_fn=<DivBackward0>)

In [29]:
model.fc1.weight.data, model.fc1.bias.data

(tensor([[0.6160, 1.7075]]), tensor([0.8573]))

In [30]:
model.fc1.weight.grad, model.fc1.bias.grad

(tensor([[38.4000, 29.2533]]), tensor([14.2667]))

# First Pass

In [31]:
x.shape, w1.shape, b1.shape

(torch.Size([3, 2]), torch.Size([1, 2]), torch.Size([1]))

In [32]:
w1 = w1.t()

In [33]:
def lin(x, w, b):
    return x @ w + b

In [40]:
x

tensor([[1.0000, 3.0000],
        [4.0000, 1.2000],
        [3.0000, 2.0000]])

In [41]:
w1

tensor([[1.],
        [2.]])

In [42]:
b1

tensor([1.])

In [34]:
fc_res = lin(x, w1, b1)
fc_res

tensor([[8.0000],
        [7.4000],
        [8.0000]])

In [35]:
fc_res.gradient

AttributeError: 'Tensor' object has no attribute 'gradient'

In [36]:
calc_loss(fc_res)

tensor([50.9200])

# Back Propogation

In [37]:
w1, b1

(tensor([[1.],
         [2.]]),
 tensor([1.]))

In [38]:
def mse_grad(inp, targ):
    inp.gradient = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [45]:
fc_res.gradient

### Gradient Self Calculation

In [46]:
fc_res.shape

torch.Size([3, 1])

In [49]:
fc_res.squeeze().shape

torch.Size([3])

In [51]:
y.shape

torch.Size([3])

In [ ]:
fc_res, y

In [64]:
2 * (fc_res.squeeze() - y).unsqueeze(-1) / fc_res.shape[0]

tensor([[4.6667],
        [4.9333],
        [4.6667]])

torch.Size([1])

In [ ]:
2 * (fc_res.squeeze() - y).unsqueeze(-1) / fc_res.shape[0]

### End of calculation

In [43]:
mse_grad(fc_res, y)

In [65]:
fc_res.gradient

tensor([[4.6667],
        [4.9333],
        [4.6667]])

In [ ]:
fc_res

In [ ]:
fc_res.shape, y.shape

In [ ]:
fc_res.gradient

In [ ]:
fc_res.gradient.shape

In [77]:
def lin_grad(inp, out, w, b):    
    # the gradient of a matrix product, is the matrix product with a transpose
#     inp.gradient = out.gradient @ w.t()
        
    w.gradient = (inp.unsqueeze(-1) * out.gradient.unsqueeze(1)).sum(0)

    b.gradient  = out.gradient.sum(0)

In [ ]:
(inp.unsqueeze(-1) * out.gradient.unsqueeze(1)).sum(0)

In [75]:
(x.unsqueeze(-1) * fc_res.gradient.unsqueeze(1)).sum(0)

tensor([[38.4000],
        [29.2533]])

In [79]:
w1.gradient

tensor([[38.4000],
        [29.2533]])

In [86]:
fc_res.gradient.sum(0)

tensor([14.2667])

In [84]:
b1.gradient

tensor([14.2667])

In [85]:
fc_res.gradient.shape

torch.Size([3, 1])

In [66]:
w1.gradient

AttributeError: 'Tensor' object has no attribute 'gradient'

In [ ]:
# lin_grad(x, fc_res, w1, b1)

In [ ]:
x.unsqueeze(-1).shape

In [ ]:
fc_res.gradient.unsqueeze(1).shape

In [ ]:
((x.unsqueeze(-1) * fc_res.gradient.unsqueeze(1))).shape

In [ ]:
(x.unsqueeze(-1) * fc_res.gradient.unsqueeze(1)).sum(0)

In [ ]:
fc_res.gradient.sum(0)

In [78]:
lin_grad(x, fc_res, w1, b1)

In [ ]:
w1.gradient, b1.shape

In [ ]:
model.fc1.weight.grad

In [ ]:
fc_res.gradient

# Re Calculate Weights

In [ ]:
lr = 0.01

In [ ]:
w1

In [ ]:
model.fc1.weight.data

In [ ]:
w1.gradient.shape

In [ ]:
w1 = w1 - lr * w1.gradient
b1 -= lr * b1.gradient

In [ ]:
model.fc1.weight.data

In [ ]:
w1

# Re Calculate Loss

In [ ]:
new_fc_res = lin(x, w1.squeeze(0), b1)
new_fc_res

In [ ]:
calc_loss(new_fc_res)

In [ ]:
w1, b1

In [ ]:
model.fc1.bias.data

# Recalculate Weights Second time

In [ ]:
train()

In [ ]:
model.fc1.weight.data

In [ ]:
model.fc1.weight.grad

In [ ]:
fc_res_2 = lin(x, w1, b1)

In [ ]:
fc_res_2.gradient

In [ ]:
mse_grad(fc_res_2, y)

In [ ]:
fc_res_2.gradient

In [ ]:
lin_grad(x, fc_res_2, w1, b1)

In [ ]:
model.fc1.weight.grad

In [ ]:
fc_res_2.gradient

In [ ]:
w1 = w1 - lr * w1.gradient
b1 -= lr * b1.gradient

In [ ]:
model.fc1.weight.data

In [ ]:
w1

In [ ]:
second_update_weights = lin(x, w1, b1)

In [ ]:
calc_loss(second_update_weights)

In [ ]:
model.fc1.weight.data

In [ ]:
w1

In [ ]:
model.fc1.bias.data

In [ ]:
b1

In [ ]:
b1.gradient

In [ ]:
model.fc1.bias.grad